In [1]:
# imports
import time
from time import sleep
import pandas as pd
from random import randint
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
# browser options
option=webdriver.ChromeOptions()
option.add_argument("--incognito")
# option.add_argument('--headless=chrome')

In [3]:
# search keyword
job_search_keyword = [
    'Data+Scientist', 'Business+Analyst', 'Data+Engineer',
    'Python+Developer' 'Full+Stack+Developer', 'Software+Engineer',
    'Machine+Learning+Engineer'
]

location_search_keyword = [
    'Kuala+Lumpur', 'Selangor', 'Petaling+Jaya',
    'Kota+Damansara', 'Shah+Alam', 'Cheras',
    'KL+Sentral', 'Puchong', 'Ampang', 'Klang'
]

pagination_url = 'https://malaysia.indeed.com/jobs?q={}&l={}&radius=15&filter=1&sort=date&start={}'

In [4]:
start = time.time()

job_ = 'Technician+Aircond'
location = 'Selangor'

job_lst = [] # store job list
job_description_list_href = []
job_description_list = []
salary_list = [] # store salary list
merged_array = [] # store merged array

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)

driver.get(pagination_url.format(job_,location,0))

# wait for a random seconds before continuing
sleep(randint(2, 6))

# find the number of jobs
p = driver.find_element(By.CLASS_NAME, 'jobsearch-JobCountAndSortPane-jobCount').text
p = p.strip(" jobs")
p = int(p.replace(',',''))
max_iter_pgs = p // 15

# set loop = 0 if only 1 page of listings
if (max_iter_pgs == 0 and p > 0):
    maxRange = 1
else:
    maxRange = max_iter_pgs

# scraper
for i in range(0,maxRange):
    driver.get(pagination_url.format(job_, location, i*10))
    
    sleep(randint(2, 4))

    job_page = driver.find_element(By.ID, "mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME, "job_seen_beacon")

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME, "jobTitle")

        job_lst.append([
            job_title.text, # job title
            job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("href"), # job link
            job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("id"), # job id
            jj.find_element(By.CLASS_NAME, "companyName").text, # company name
            jj.find_element(By.CLASS_NAME, "companyLocation").text, # company location
            jj.find_element(By.CLASS_NAME, "date").text # date posted
        ])

        try:
            salary_list.append(jj.find_element(By.CLASS_NAME, "salary-snippet-container").text)
        except NoSuchElementException:
            salary_list.append(None)

        try:
            pop_up_button = jj.find_element(By.XPATH, '//*[@id="mosaic-desktopserpjapopup"]/div[1]/button')
            pop_up_button.click()
        except NoSuchElementException:
            pass

        # pop_up_button.click()

        # job_title.click()

        # sleep(randint(3, 5)) # might need to change timing

        # try:
        #     job_description_list.append(driver.find_element(By.ID,"jobDescriptionText").text)
        # except:
        #     job_description_list.append(None)

# extract job description links
for i in range(len(job_lst)):
    job_description_list_href.append(job_lst[i][1])

# extract job description from each link
for i in job_description_list_href:
    driver.get(i)
    
    sleep(randint(3, 5))

    try:
        job_description_list.append(driver.find_element(By.ID,"jobDescriptionText").text)
    except:
        job_description_list.append(None)

driver.quit()
end = time.time()

# merge job listing and with corresponding salary
for job, salary, desc in zip(job_lst, salary_list, job_description_list):
    merged_array.append(job + [salary] + [desc])

# print scraping details
print(end - start, 'seconds to complete action!')
print('---------------------')
print('Max Iterable Pages for this search:', max_iter_pgs)
print('Job Count:', p)

328.64494609832764 seconds to complete action!
---------------------
Max Iterable Pages for this search: 3
Job Count: 45


## Testing

In [5]:
len(merged_array)

45

## Pandas

In [8]:
df = pd.DataFrame(merged_array, columns = [
    'Job Title', 'Job Link', 'Job ID',
    'Company', 'Location', 'Job Posting',
    'Salary', 'Job Description'
])

# change this path
df.to_excel('..//Indeed-WS//excel//results.xlsx', 
            sheet_name='Sheet1', index=False)